In [ ]:
# Import dati dalla prof
from scipy.io import loadmat
import numpy as np
import scipy.linalg as spl
import RisolviSis as RS
import matplotlib.pyplot as plt
dati = loadmat('testE.mat')
A=dati["A"] 
b=dati["b"]


# m = n ?

In [ ]:
# Innanzittutto devo verificare se il sistema è sovradeterminato o meno
# m =/= n il sistema è sovradeterminato.
# Inoltre in base ai valori sulle dimensioni posso anche capire quanto sia
# grande o meno la matrice e decidere meglio quale algortimo applicare.
n, m = A.shape
print("Dimensione di A:", n, m)


# Densa o Sparsa ?
Questo lo faccio solo se m = n.

In [ ]:
# Se ho una matrice quadrata controllo se questa sia densa o sparsa:
# se più del 33% degli elementi è diverso da 0 allora la matrice è densa.
n_zeri = np.count_nonzero(A) / (n * m)
perc_n_zeri = n_zeri * 100
print("Percentuale elementi diversi da zero:", perc_n_zeri,"%")


# Simmetrica ?
Solo se la matrice è grande e sparsa: in questo caso usi i metodi iterativi.

In [ ]:
# Se ho una matrice quadrata e grande e sparsa, verifico ora che la matrice sia 
# simmetrica oppure no: in base a questa valutazione capisco quale metodo 
# utilizzare.
# Se la matrice è uguale alla trasposta allora questa sarà definita positiva.
flag = (A == A.T)
if (np.all(flag) == False):
    print("La matrice non è simmetrica")
else:
    print("La matrice è simmetrica e definita positiva")


# Simmetrica == True?
GS SOR, Gradiente, Gradiente coniugato.

# Simmetrica == False?
Controllo se la diagonale è dominante.

# Diagonale dominante?
Jacobi, Gauss-Siedel, GS SOR.

In [ ]:
# A questo punto passo ad analizzare la diagonale della matrice: da questo 
# capisco se la diagonale è dominante o meno e se lo è anche in modo stretto.
# A e' a diagonale dominante se il valore assoluto dell'elemento sulla diagonale 
# e' >= della somma in valore in valore assoluto di tutti i valori sulla
# rispettiva riga.
def check_diagonale(A):
    n = A.shape[0]
    flag = True
    for i in range(n):
       diag_elem = np.abs(A[i, i])
       print("elemento diagonale:", diag_elem)
       # per tutte le righe: elementi in riga sommati - elemento diagonale.
       row_sum = np.sum(np.abs(A[i, :])) - np.abs(A[i, i])
       if diag_elem < row_sum:
            print("Matrice non a diagonale dominante")
            flag = False
            return flag
    
    # Se e' a diagonale dominante hai: Jacobi, Gauss-Siedel e GS SOR
    return flag

diag = check_diagonale(A)
print("Matrice a diagonale dominante? ", diag)

# Piccola e Densa ?
Ti affidi ai metodi di fattorizzazione:
- QR: sempre possibile.
- LU: determinante di A diverso da 0.
- Cholesky: esiste per matrici simmetriche e definite positive.

In [ ]:
# Vedo se il determinante di A è diverso da 0
rank = np.linalg.matrix_rank(A)
if(A.shape[0] == rank):
    print("Rango massimo e il determinante è diverso da 0.")
else:
    print("Rango non massimo e il determinante è nullo.")


# Simmetria ?

In [ ]:
# Se la matrice è uguale alla trasposta allora questa sarà definita positiva.
flag = (A == A.T)
if (np.all(flag) == False):
    print("La matrice non è simmetrica")
else:
    print("La matrice è simmetrica e definita positiva")


## Esercizio 2 
Si consideri il sistema lineare Ax=b, con A matrice e b termine noto memorizzati nel file ``testF.mat``.Risolvere il sistema utilizzando il metodo più adatto. Commentare i risultati ottenuti  giustificandoli alla luce della teoria. 

Per la lettura dei dati procedere nel seguente modo:

``from scipy.io import loadmat``

``import numpy as np``

``dati = loadmat('testF.mat')``

``A=dati["A"] ``

`` b=dati["b"] ``

In [ ]:
import scipy.linalg as spLin
dati = loadmat('testF.mat')
A1=dati["A"] 
b1=dati["b"]

In [ ]:
m1,n1=A1.shape
print("Dimensione di A",m1,n1)

Si tratta di un sistema sovraderminato. Verifico se la matrice è a Rango massimo facendo uso della SVD e contando il numero di valori singolari diversi da zero.

In [ ]:
 U,s,VT=spLin.svd(A1)  #Attenzione : Restituisce U, Sigma e VT=VTrasposta)
 thresh= np.spacing(1)*m*s[0] #m=max(m,n)
 k=np.count_nonzero(s>thresh) #Calcolo del rango della matrice, numero dei valori singolari diversi maggiori della soglia
 print("rango=",k)
 if k<n1 :
    print("La matrice non ha rango massimo")
 else:
    print("La matrice ha rango massimo")


La matrice non è a rango massimo. Utilizzo il metodo SVD per risolvere il sistema sovradeterminato.

In [ ]:
def SVDLS(A,b):
    n=A.shape[1]  # numero di colonne di A
    U,s,VT=spLin.svd(A)  #Attenzione : Restituisce U, Sigma e VT=VTrasposta)
    V=VT.T
     
    thresh=np.spacing(1)*m*s[0] 
    ##Calcolo del rango della matrice, numero dei valori singolari maggiori di una soglia
    k=np.count_nonzero(s>thresh)
    print("rango=",k)
    if k<n:
        print("La matrice non è a rango massimo")
    else:
        print("La matrice è a rango massimo")
    d=U.T@b
    d1=d[:k].reshape(k,1)
    s1=s[:k].reshape(k,1)
    #Risolve il sistema diagonale di dimensione kxk avene come matrice dei coefficienti la matrice Sigma
    c=d1/s1
    x=V[:,:k]@c 
    residuo=np.linalg.norm(d[k:])**2
    return x,residuo
     

In [ ]:
x,residuo=SVDLS(A1,b1)
print("Soluzione nel senso dei minimi quadrati\n  ", x)
print("residuo ",residuo)
print("Norma soluzione ", np.linalg.norm(x))

In [ ]:
def qrLS(A,b):
    n=A.shape[1]  # numero di colonne di A
    Q,R=spLin.qr(A)
    h=Q.T@b
    x,flag=RS.Usolve(R[0:n,:],h[0:n])
    residuo=np.linalg.norm(h[n:])**2
    return x,residuo

In [ ]:
def eqnorm(A,b):
 
    G=A.T@A
     
    print("Indice di condizionamento di G ",np.linalg.cond(G))
    f=A.T@b
    L=spLin.cholesky(G,lower=True)
   
    y,flag=RS.Lsolve(L,f)
    if flag==0:
        x,flag=RS.Usolve(L.T,y)
    
    
    return x

## Esercizio 3

Caricare dal  file  Approx_C.mat  le coppie di dati sperimentali (x(i),y(i)),i=1,M) e visto il trend dei dati, scegliere opportunamente  il grado del polinomio con cui approssimare  i dati. Costruirlo con un metodo di approssimazione nel senso dei minimi quadrati a scelta tra quello più opportuno alla luce della teoria, calcolare la norma 2 al quadrato del residuo e visualizzare  il grafico.


In [ ]:
import scipy.linalg as spLin
dati = loadmat('Approx_C.mat')

x=dati["x"]
y=dati["y"]
m=x.shape[0]
x=x.reshape(m,)
y=y.reshape(m,)
plt.plot(x,y,'ro')
 
#Visualizzando il trend dei dati osservo che hanno un andamento di un polinomio cubico
n=3
n1=n+1
A3=np.vander(x,increasing=True)[:,:n1]
print('Rango ' , np.linalg.matrix_rank(A3),'Condizionamento ',np.linalg.cond(A3))
#La matrice A è a rango massimo e mediamente mal condizionata,
#la matrice A^T A, avraà indice di condizionamento il quadrato dell'indice di condizionamento
#di A, per cui uso il metodo QR
alphaqr,res =qrLS(A3,y )
print("residuo ",res)
xx=np.linspace(np.min(x),np.max(x),200)
polQR=np.polyval(np.flip(alphaqr),xx)
plt.plot(xx,polQR)


## Esercizio 4


Siano assegnati i punti del piano di coordinate $(x_i, y_i)$, $i=1,\ldots,4$ con 
$$
\begin{array}{c}
x_1=0, \quad x_2=4, \quad x_3=0, \quad x_4=5\\
y_1=0, \quad y_2=0, \quad y_3=4, \quad y_4=6.
\end{array}
$$ 


 si costruisca il sistema lineare sovradeterminato ottenuto imponendo il passaggio della circonferenza di equazione $x^2+y^2+a_1x+a_2y+a_3=0$ per i quattro punti assegnati, e si denotino con ${\bf A}$ e ${\bf b}$ la matrice dei coefficienti e il termine noto ottenuti;

si sfrutti il metodo  più opportuno per ottenere la soluzione ${\bf a}^*$ del problema ai minimi quadrati $\min_{{\bf a} \in \mathbb{R}^3} \|{\bf A}{\bf a}-{\bf b}\|_2^2$;

si calcoli il valore di $\|{\bf A}{\bf a}^*-{\bf b}\|_2^2$ e, dopo averne determinato centro e raggio, si rappresentino in una stessa figura la circonferenza di equazione $x^2+y^2+a^*_1x+a^*_2y+a^*_3=0$ (individuata dalla soluzione ${\bf a}^*$ del problema ai minimi quadrati) e i quattro punti assegnati dal problema.

Si ricorda che data una circonferenza nella forma canonica
$$ x^2+y^2+a_1 x+a_2 x+a_3=0$$

le coordinate del suo centro sono 
$$C≡(-a_1/2,-a_2/2)$$

e la lunghezza del suo raggio è data da:

$$r=\sqrt{\frac{a_1^2}{4}+\frac{a_2^2}{4} -a_3}$$

e la circoferenza può essere rappresentata in forma parametrica $C(t),  t \in [0, 2 \cdot \pi]$ e le due componenti parametriche sono $$x(t)= C_0+r\cdot cos(t);$$ $$ y(t)= C_1+r\cdot sin(t)$$

In [ ]:
import math
px=np.array([0,4,0,5])
py=np.array([0,0,4,6])
#Costruisco il sistema lineare sovradeterminato come richiesto
A=np.array([[0,0,1],[4,0,1],[0,4,1],[5,6,1]])
b=np.array([[0],[-16],[-16],[-61]])
 
a,err=SVDLS(A,b)
print("Norma 2 al quadrato dell'errore ",err)
#Calcolo il centro
cx=-a[0]/2
cy=-a[1]/2
#Calcolo il raggio con la formula data
r1=math.sqrt((a[0]**2)/4+(a[1]**2)/4-a[2])
t=np.linspace(0,2*math.pi,100)
#Costruisco le due componenti parametriche della circonferemza
x=cx+r1*np.cos(t)
y=cy+r1*np.sin(t)
plt.plot(x,y,'r-')
plt.plot(px,py,'go')
plt.axis('equal')

## Esercizio 5

Per i dati $(x_i, y_i)$ riportati nei seguenti array

x = np.array([0.0004, 0.2507, 0.5008, 2.0007, 8.0013]) 

y = np.array([0.0007,0.0162, 0.0288, 0.0309, 0.0310])

- costruire la retta di regressione;
- costruire la parabola approssimante i dati nel senso dei minimi quadrati;
- determinare l'approssimazione ai minimi quadrati espressa in termini di
basi esponenziali: $y = a + b e^{-x}+ c e^{-2x}$

Quale tra le tre approssimazioni risulta la migliore? (Confrontare i gra
ci e la norma euclidea al quadrato del vettore dei residui).

In [ ]:
m=5
x4 = np.array([0.0004, 0.2507, 0.5008, 2.0007, 8.0013]) 
y4 = np.array([0.0007,0.0162, 0.0288, 0.0309, 0.0310]) 
M=np.zeros((5,3))
M[:,0]=np.ones((5,))
M[:,1]=np.exp(-x4)
M[:,2]=np.exp(-2*x4)

In [ ]:
print('Rango di ---->' , np.linalg.matrix_rank(M))
print("Con Condizionamento M --> ",np.linalg.cond(M))
#La matrice M è a rango massimo, ha condizionamento 18.45, quindi la marice G delle equazioni
#normali (di dimensioni 4x4) avrebbe indice
#di condizionamento circa 343. Utilizzo il metodo 
aexp,resexp=qrLS(M,y4)


In [ ]:
xx=np.linspace(np.min(x4),np.max(x4),200)
polexp=aexp[0]+aexp[1]*np.exp(-xx)+aexp[2]*np.exp(-2*xx)

# Calcolo del polinomio approssimante di grado 1
n=1
n1=n+1
A1=np.vander(x4,increasing=True)[:,:n1]
print('Rango di A1 --->',np.linalg.matrix_rank(A1))
print("Condizionamento di A1---> ",np.linalg.cond(A1))
#Matrice a rango massimo e ben condizionata: uso le equazioni normali
alpha1=eqnorm(A1,y4)
pol1=np.polyval(np.flip(alpha1),xx)

#Parabola approssimante
n=2
n1=n+1
A2=np.vander(x4,increasing=True)[:,:n1]
print('Rango di A2',np.linalg.matrix_rank(A2))
print("Condizionamento di A2 ",np.linalg.cond(A2))
#Matrice a rango massimo e mediamente mal condizionata: uso qr
alpha2,res2=qrLS(A2,y4)
pol2=np.polyval(np.flip(alpha2),xx)
print("quadrato residuo exp",resexp)
print("quadrato residuo pol grado 2 ",res2)

plt.plot(x4,y4,'ro',xx,polexp,'b--',xx,pol1,'g:',xx,pol2,'m-')
plt.legend(['Osservazioni','Esponenziale','Polinomio grado 1','Polinomio grado 2'])